In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
from tqdm import tqdm

In [146]:
processed_df = pd.read_csv("../data/processed_dhq_data.csv")

In [148]:
processed_df['DHQarticle-id'] = processed_df['DHQarticle-id'].astype(str)
processed_df['DHQarticle-id'] = processed_df['DHQarticle-id'].str.zfill(6)

,DHQarticle-id,volume,issue,articleType,date_when,dhq_keywords,language_ident,dhq_abstract,file_name,title,...,body_text,date_processed,issue_title,scraped_editors,scraped_authors,article_link,article_title,abstract,issue_text,issue_link


In [131]:
initial_df = pd.read_csv("../data/initial_dhq_data.csv")

In [136]:
initial_df.issue.unique()

array(['1', '2', '3', '4', 'yet to be assigned'], dtype=object)

In [133]:
article_links_df = pd.read_csv("../data/dhq_article_links.csv")
pattern = r'.*vol/([^/]*)/([^/]*)/(\d+)/\3\.html'
article_links_df[['volume', 'issue', 'DHQarticle-id']] = article_links_df.article_link.str.extract(pattern)

In [143]:
# add zeroes to volume so always three digits
article_links_df = article_links_df.rename(columns={'authors': 'scraped_authors', 'editors': 'scraped_editors'})
initial_df['volume'] = initial_df['volume'].astype(str)
initial_df['issue'] = initial_df['issue'].astype(str)
initial_df['DHQarticle-id'] = initial_df['DHQarticle-id'].astype(str)
article_links_df['volume'] = article_links_df['volume'].astype(str)
article_links_df['issue'] = article_links_df['issue'].astype(str)
article_links_df['DHQarticle-id'] = article_links_df['DHQarticle-id'].astype(str)
merged_df = pd.merge(initial_df, article_links_df, on=['DHQarticle-id', 'volume', 'issue'], how='left')
# merged_df.to_csv("../data/processed_dhq_data.csv", index=False)

In [145]:
merged_df[(merged_df.volume != "yet to be assigned") & (merged_df.article_link.isna())]

,DHQarticle-id,volume,issue,articleType,date_when,dhq_keywords,language_ident,dhq_abstract,file_name,title,...,body_text,date_processed,issue_title,scraped_editors,scraped_authors,article_link,article_title,abstract,issue_text,issue_link
9,000009_01,001,2,article,12 September 2007,DHQ classification scheme; full list available...,en,Because so little primary historical work has ...,../data/dhq-journal/articles/000009/000009_01.xml,Somewhere Nearby is Colossal Cave: Examining W...,...,Introduction\n Will Crowther's Colo...,2007-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,000164_01,007,2,opinion,12 October 2013,DHQ classification scheme; full list available...,en,"Within the digital humanities, there are many ...",../data/dhq-journal/articles/000164/000164_01.xml,FairCite,...,FairCite: Practical Steps towards a Fairer Cul...,2013-10-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,000214,009,3,article,23 December 2015,DHQ classification scheme; full list available...,en,Academic writing has generally been understood...,../data/dhq-journal/articles/000214/000214.xml,Multimodal Authoring and Authority in Educatio...,...,Creator's Statement\n \n\n\n ...,2015-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,000432,013,4,article,NaN,DHQ classification scheme; full list available...,en,Skillful identification and interpretation of ...,../data/dhq-journal/articles/000432/000432.xml,Finding and Interpreting Arguments: An Importa...,...,Prologue: From Distant Reading to Close Readin...,2019-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
449,000424,014,2,article,NaN,DHQ classification scheme; full list available...,en,Estudos recentes estão conseguindo demonstrar ...,../data/dhq-journal/articles/000424/000424.xml,SIG e história da arquitetura. Avances no\n ...,...,Introdução\n Os Sistemas de Inf...,2020-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,000580,014,3,article,NaN,DHQ classification scheme; full list available...,en,This article looks at online history projects ...,../data/dhq-journal/articles/000580/000580.xml,"Lost Spaces, Lost\n Technologies...",...,What does it mean to be lost?\n \n ...,2020-09-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469,000463_01,014,3,article,01 September 2020,DHQ classification scheme; full list available...,en,What values does infrastructure-building repre...,../data/dhq-journal/articles/000463/000463_01.xml,Scholarly Infrastructure as Critical Argument:...,...,We live in a world connected not merely by tec...,2020-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
510,000488,015,1,article,NaN,DHQ classification scheme; full list available...,en,NaN,../data/dhq-journal/articles/000488/000488.xml,From the Presupposition of Doom to the Manifes...,...,Sample Embedded GISST\n \n ...,2021-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,000573,015,1,article,NaN,DHQ classification scheme; full list available...,en,"Many tools can produce\n maps,...",../data/dhq-journal/articles/000573/000573.xml,"Networks, Maps, and Time: Visualizing Historic...",...,Introduction\n Data visualization is a...,2021-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,000575,015,3,article,"November 19, 2021",DHQ classification scheme; full list available...,en,A book review of Hannu Salmi’s,../data/dhq-journal/articles/000575/000575.xml,The Age Old Question: A Review of What is Digi...,...,The latest addition to the What is History? se...,2021-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
article_links_df.volume.unique()

array(['017', '016', '015', '014', '013', '012', '011', '010', '009',
       '008', '007', '006', '005', '004', '003', '002', '001'],
      dtype=object)

In [70]:
# Define the XML namespaces
namespaces = {
    'tei': "http://www.tei-c.org/ns/1.0",
    'dhq': "http://www.digitalhumanities.org/ns/dhq",
    'xml': "http://www.w3.org/XML/1998/namespace"
}
directory_path = "../data/dhq-journal/articles"
all_data = []
# xml_files = [os.path.join(directory_apth, file_name) for file_name in os.listdir(directory_apth) if file_name.endswith(".xml")]

exclude = ['old', 'converted', 'dhq', 'sample', 'recovered', 'test', 'walsh']

# List only XML files
xml_files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(directory_path) for f in filenames 
             if f.endswith('.xml') 
             and not any(ex_str in f for ex_str in exclude) 
             and not f.startswith('999') and not '_' in f]

len(xml_files)

728

In [86]:
def process_xml_files(xml_files, output_path):
    if os.path.exists(output_path):
        final_df = pd.read_csv(output_path)
    else:
        for file_name in tqdm(xml_files, desc="Processing XML files"):
        
            with open(file_name, 'r', encoding='utf-8') as f:
                file_content = f.read().strip()
                
            if file_content == '<?xml version="1.0" encoding="UTF-8"?>':
                continue  # Skip to the next file if the content is just the XML declaration
            
            try:
                tree = ET.parse(file_name)
                root = tree.getroot()
            except ET.ParseError:
                print(f"Error parsing {file_name}. Skipping...")
                continue

            # Extracting the required data with checks to avoid errors
            base_data = {
                'DHQarticle-id': root.find(".//tei:publicationStmt/tei:idno[@type='DHQarticle-id']", namespaces=namespaces).text if root.find(".//tei:publicationStmt/tei:idno[@type='DHQarticle-id']", namespaces=namespaces) is not None else None,
                'volume': root.find(".//tei:publicationStmt/tei:idno[@type='volume']", namespaces=namespaces).text if root.find(".//tei:publicationStmt/tei:idno[@type='volume']", namespaces=namespaces) is not None else None,
                'issue': root.find(".//tei:publicationStmt/tei:idno[@type='issue']", namespaces=namespaces).text if root.find(".//tei:publicationStmt/tei:idno[@type='issue']", namespaces=namespaces) is not None else None,
                'articleType': root.find(".//tei:publicationStmt/dhq:articleType", namespaces=namespaces).text if root.find(".//tei:publicationStmt/dhq:articleType", namespaces=namespaces) is not None else None,
                'date_when': root.find(".//tei:publicationStmt/tei:date", namespaces=namespaces).text if root.find(".//tei:publicationStmt/tei:date", namespaces=namespaces) is not None else None,
                'dhq_keywords': root.find(".//tei:encodingDesc/tei:classDecl/tei:taxonomy[@xml:id='dhq_keywords']/tei:bibl", namespaces=namespaces).text if root.find(".//tei:encodingDesc/tei:classDecl/tei:taxonomy[@xml:id='dhq_keywords']/tei:bibl", namespaces=namespaces) is not None else None,
                'language_ident': root.find(".//tei:profileDesc/tei:langUsage/tei:language", namespaces=namespaces).attrib['ident'] if root.find(".//tei:profileDesc/tei:langUsage/tei:language", namespaces=namespaces) is not None else None,
                'dhq_abstract': root.find(".//tei:text/tei:front/dhq:abstract/tei:p", namespaces=namespaces).text if root.find(".//tei:text/tei:front/dhq:abstract/tei:p", namespaces=namespaces) is not None else None,
                'file_name': file_name
            }

            # Extract title
            title_element = root.find(".//tei:titleStmt/tei:title", namespaces=namespaces)
            if title_element is not None:
                # Concatenate all text and tail components of the element and its descendants
                title_parts = [title_element.text] + [e.text + (e.tail if e.tail else "") for e in title_element.findall(".//")]
                base_data['title'] = "".join(filter(None, title_parts))



            # Extract author information
            author_elements = root.findall(".//tei:titleStmt/dhq:authorInfo", namespaces=namespaces)

            authors_data = []

            for author_element in author_elements:
                author_data = {}
                
                # Extract author name
                author_name_element = author_element.find("dhq:author_name", namespaces=namespaces)
                if author_name_element is not None:
                    first_name = author_name_element.text
                    last_name_element = author_name_element.find("dhq:family", namespaces=namespaces)
                    if last_name_element is not None:
                        full_name = f"{first_name} {last_name_element.text}".strip()
                    else:
                        full_name = first_name
                    author_data['author_name'] = full_name

                # Extract affiliation
                affiliation_element = author_element.find("dhq:affiliation", namespaces=namespaces)
                if affiliation_element is not None:
                    author_data['affiliation'] = affiliation_element.text

                # Extract email
                email_element = author_element.find("email", namespaces=namespaces)
                if email_element is not None:
                    author_data['email'] = email_element.text

                # Extract bio
                bio_element = author_element.find("dhq:bio/tei:p", namespaces=namespaces)
                if bio_element is not None:
                    author_data['bio'] = ''.join(bio_element.itertext()).strip()
                
                authors_data.append(author_data)

            base_data['authors'] = authors_data

            # Extracting paragraphs from the body
            # Check if paragraphs are inside a <div> tag
            # Get the <body> element
            body_element = root.find(".//tei:text/tei:body", namespaces=namespaces)

            # Extract all text from the <body> element and its descendants
            body_text = ''.join(body_element.itertext()).strip()

            base_data['body_text'] = body_text

            # Then, instead of creating a separate dataframe for paragraphs, you can directly append the base_data dictionary to the all_data list:
            data_df = pd.DataFrame([base_data])

            all_data.append(data_df)

        # Convert the data list to a DataFrame
        final_df = pd.concat(all_data)
        final_df.to_csv(output_path, index=False)
    return final_df

In [87]:

directory_path = "../data/dhq-journal/articles"
all_data = []
# xml_files = [os.path.join(directory_apth, file_name) for file_name in os.listdir(directory_apth) if file_name.endswith(".xml")]

exclude = ['old', 'converted', 'dhq', 'sample', 'recovered', 'test', 'walsh']

# List only XML files
xml_files = [os.path.join(dp, f) for dp, _, filenames in os.walk(directory_path) for f in filenames 
             if f.endswith('.xml') 
             and not any(ex_str in f for ex_str in exclude) 
             and not f.startswith('999') and not f.startswith('000000') and not '_' in f]


df = process_xml_files(xml_files, '../data/initial_dhq_data.csv')

Processing XML files: 100%|██████████| 727/727 [00:01<00:00, 427.89it/s]


In [88]:
df = df.sort_values(by=['DHQarticle-id'])

In [75]:
df[df.volume.isna()]['DHQarticle-id'].unique()

array(['000657', '000664', '000716', '000718', '000723', None],
      dtype=object)

In [8]:
df[df['DHQarticle-id'].isna()]

,DHQarticle-id,volume,issue,articleType,date_when,dhq_keywords,language_ident,dhq_abstract,file_name,title,authors,body_text
0,None,None,None,article,None,DHQ classification scheme; full list available...,en,The project Untangling the cordel (2020-2024) ...,../data/dhq-journal/articles/000695/000695.xml,,"[{'author_name': 'Elina Leblanc', 'affiliatio...","Introduction\n\t\t\t\t\tIn a recent article, C..."


In [25]:
df[df.date_when.isna()]['DHQarticle-id'].unique()

array(['000424', '000432', '000488', '000573', '000580', '000646',
       '000664', '000673', '000680', '000709', '000714', '000716',
       '000718', '000720', '000723', '000729'], dtype=object)

In [50]:
df[df['DHQarticle-id'] == "000424"]

,DHQarticle-id,volume,issue,articleType,date_when,dhq_keywords,language_ident,dhq_abstract,file_name,title,authors,body_text,date_processed
449,000424,014,2,article,None,DHQ classification scheme; full list available...,en,Estudos recentes estão conseguindo demonstrar ...,../data/dhq-journal/articles/000424/000424.xml,SIG e história da arquitetura. Avances no\n ...,[{'author_name': 'Patricia ...,Introdução\n Os Sistemas de Inf...,2020-08-15


In [28]:
df.reset_index(drop=True, inplace=True)

In [89]:
df['DHQarticle-id'] = df['DHQarticle-id'].str.strip()

In [90]:
df.date_when = df.date_when.str.replace('Feburary', 'February')
df['date_processed'] = pd.to_datetime(df['date_when'])

In [91]:
def check_duplicates(rows):
    if len(rows)> 1:
        rows = rows.sort_values(by=['date_when'], ascending=False)
        rows = rows[0:1]
    return rows

df = df.groupby('DHQarticle-id').apply(check_duplicates)

In [92]:
df.loc[df.file_name == "../data/dhq_data/000664.xml", "volume"] = "016"
df.loc[df.file_name == "../data/dhq_data/000664.xml", "issue"] = "4"

In [93]:
df.loc[df.volume.isna(), 'volume'] = 'yet to be assigned'
df.loc[df.issue.isna(), 'issue'] = 'yet to be assigned'

In [94]:
def infer_dates(rows):
    missing_dates = rows[rows.date_processed.isna()]
    has_dates = rows[rows.date_processed.notna()]
    if len(has_dates) == 0:
        rows.date_processed = None
        return rows
    latest_date = has_dates.date_processed.max()
    missing_dates.date_processed = latest_date
    rows = pd.concat([missing_dates, has_dates])
    return rows

In [95]:
df = df.groupby(['volume', 'issue']).apply(infer_dates)
df = df.reset_index(drop=True)

In [107]:
test = pd.read_csv("../data/dhq_article_links.csv")

In [84]:
test['volume'] = test.article_link.str.split('vol').str[-1].str.split('/').str[1]
test['issue'] = test.article_link.str.split('vol').str[-1].str.split('/').str[2]
test['DHQarticle-id'] = test.article_link.str.split('vol').str[-1].str.split('/').str[3]

In [109]:
pattern = r'.*vol/([^/]*)/([^/]*)/(\d+)/\3\.html'
test[['volume', 'issue', 'DHQarticle-id']] = test.article_link.str.extract(pattern)

In [112]:
test['DHQarticle-id'] = test['DHQarticle-id'].astype(str)

In [113]:
test.to_csv("../data/dhq_article_links.csv", index=False)

In [115]:
test.columns

Index(['issue_title', 'editors', 'authors', 'article_link', 'article_title',
       'abstract', 'issue_text', 'issue_link', 'volume', 'issue',
       'DHQarticle-id'],
      dtype='object')

In [116]:
df.columns

Index(['DHQarticle-id', 'volume', 'issue', 'articleType', 'date_when',
       'dhq_keywords', 'language_ident', 'dhq_abstract', 'file_name', 'title',
       'authors', 'body_text', 'date_processed'],
      dtype='object')

In [101]:
df[~df['DHQarticle-id'].isin(test['DHQarticle-id'])]

,DHQarticle-id,volume,issue,articleType,date_when,dhq_keywords,language_ident,dhq_abstract,file_name,title,authors,body_text,date_processed
0,000001,001,1,review,3 April 2007,DHQ classification scheme; full list available...,en,A review of Willard McCarty's,../data/dhq-journal/articles/000001/000001.xml,Philosophy and Digital Humanities: A review of...,"[{'author_name': 'Johanna Drucker', 'affiliat...",Humanities computing is still a fledgling disc...,2007-04-03
1,000002,001,1,article,3 April 2007,DHQ classification scheme; full list available...,en,"In this paper, I extend theoretical understand...",../data/dhq-journal/articles/000002/000002.xml,Interpretative Quests in Theory and Pedagogy,"[{'author_name': 'Jeff Howard', 'affiliation'...",Quests: Theoretical Background\n A ...,2007-04-03
2,000003,001,1,article,3 April 2007,DHQ classification scheme; full list available...,None,Lincoln/Net (,../data/dhq-journal/articles/000003/000003.xml,\n Webs of Significance: The Abr...,"[{'author_name': 'Drew VandeCreek', 'affiliat...",I\n Lincoln/Net (http://lincoln.lib...,2007-04-03
3,000004,001,1,article,3 April 2007,DHQ classification scheme; full list available...,en,"For over a thousand years, Tibet has preserved...",../data/dhq-journal/articles/000004/000004.xml,Encoding for Endangered Tibetan Texts,"[{'author_name': 'Linda E. Patrik', 'affiliat...","For over a thousand years, Tibet has preserved...",2007-04-03
4,000005,001,1,article,3 April 2007,DHQ classification scheme; full list available...,en,Recent efforts to reconceptualize text analysi...,../data/dhq-journal/articles/000005/000005.xml,Reading Potential: The Oulipo and the Meaning ...,"[{'author_name': 'Mark Wolff', 'affiliation':...",Researchers in the humanities who focus their ...,2007-04-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,000721,017,4,article,15 December 2023,DHQ classification scheme; full list available...,en,Digital Humanities pedagogy seminars (DHPSs) a...,../data/dhq-journal/articles/000721/000721.xml,What Counts? Digital Humanities Pedagogy Semin...,"[{'author_name': 'Crystal Hall', 'affiliation...",Digital Humanities pedagogy seminars (DHPSs) a...,2023-12-15
720,000664,yet to be assigned,yet to be assigned,article,None,DHQ classification scheme; full list available...,en,"The annual, international Digital Humanities c...",../data/dhq-journal/articles/000664/000664.xml,Response to\n\t\t\t\t\t\tThe circus we deserve...,[{'author_name': 'The Alliance of Digital Huma...,Executive Summary \n\t\t\t\t\tWe thank DHQ for...,NaT
721,000716,yet to be assigned,yet to be assigned,article,None,DHQ classification scheme; full list available...,en,Late Latin literature is characterized by nume...,../data/dhq-journal/articles/000716/000716.xml,Towards a\n differentiated digit...,"[{'author_name': 'Barbara Feichtinger', 'affi...",A central goal of the DFG project Quoting as a...,NaT
722,000718,yet to be assigned,yet to be assigned,article,None,DHQ classification scheme; full list available...,en,"In this essay, I give a labor history of the ...",../data/dhq-journal/articles/000718/000718.xml,Fingerprints of British Book History: A Femini...,"[{'author_name': 'Ana Quiring', 'affiliation':...","Upon its founding in 2002, Google Books promis...",NaT


In [54]:
df[df.volume == "yet to be assigned"]

,DHQarticle-id,volume,issue,articleType,date_when,dhq_keywords,language_ident,dhq_abstract,file_name,title,authors,body_text,date_processed
724,000664,yet to be assigned,yet to be assigned,article,None,DHQ classification scheme; full list available...,en,"The annual, international Digital Humanities c...",../data/dhq-journal/articles/000664/000664.xml,Response to\n\t\t\t\t\t\tThe circus we deserve...,[{'author_name': 'The Alliance of Digital Huma...,Executive Summary \n\t\t\t\t\tWe thank DHQ for...,NaT
725,000716,yet to be assigned,yet to be assigned,article,None,DHQ classification scheme; full list available...,en,Late Latin literature is characterized by nume...,../data/dhq-journal/articles/000716/000716.xml,Towards a\n differentiated digit...,"[{'author_name': 'Barbara Feichtinger', 'affi...",A central goal of the DFG project Quoting as a...,NaT
726,000718,yet to be assigned,yet to be assigned,article,None,DHQ classification scheme; full list available...,en,"In this essay, I give a labor history of the ...",../data/dhq-journal/articles/000718/000718.xml,Fingerprints of British Book History: A Femini...,"[{'author_name': 'Ana Quiring', 'affiliation':...","Upon its founding in 2002, Google Books promis...",NaT
727,000723,yet to be assigned,yet to be assigned,article,None,DHQ classification scheme; full list available...,en,The presented article proposes that one of the...,../data/dhq-journal/articles/000723/000723.xml,Problems of Authorship Classification: Recogni...,"[{'author_name': 'František Válek', 'affiliat...",Introduction\n Authorship attributi...,NaT


In [143]:
final_df.loc[final_df.file_name == "../data/dhq_data/000664.xml", "volume"] = "016"
final_df.loc[final_df.file_name == "../data/dhq_data/000664.xml", "issue"] = "4"
final_df.loc[final_df.file_name == "../data/dhq_data/000684.xml", "volume"] = "017"
final_df.loc[final_df.file_name == "../data/dhq_data/000684.xml", "issue"] = "2"
final_df.date_when = final_df.date_when.str.replace('Feburary', 'February')

In [144]:
final_df['volume'] = final_df.volume.astype(int)
final_df['issue'] = final_df.issue.astype(int)

In [145]:
final_df.isna().sum()

DHQarticle-id      0
volume             0
issue              0
articleType        0
date_when         12
dhq_keywords       0
language_ident     2
dhq_abstract      21
file_name          0
title              0
authors            0
body_text          0
dtype: int64

In [146]:
older_df.isna().sum()

DHQarticle-id      0
volume            15
issue             15
articleType        0
date              18
dhq_keywords       0
language_ident     2
dhq_abstract      24
file_name          0
title              0
authors            0
body_text          0
dtype: int64

In [147]:
older_df = older_df.rename(columns={'date_when': 'date'})

In [148]:
merged_df = pd.merge(final_df, older_df[['DHQarticle-id', 'date']], on='DHQarticle-id', how='outer')
merged_df = merged_df[merged_df.body_text.notna()]

In [149]:
merged_df.loc[(merged_df.date_when != merged_df.date) & (merged_df.date_when.isna()), 'date_when'] = merged_df.date

In [150]:
missing = merged_df.loc[(merged_df.date_when != merged_df.date) & (merged_df.date_when.isna())]

In [151]:
xml_links = pd.read_csv("../data/dhq_xml_links.csv")

In [152]:
xml_links['file_name'] = "../data/dhq_data/" + xml_links.xml_link.str.split('/').str[-1]

In [153]:
xml_links[xml_links.file_name.isin(missing.file_name)]

,xml_link,volume,issue,DHQarticle-id,file_name
46,/dhq/vol/17/2/000673.xml,17,2,673,../data/dhq_data/000673.xml
47,/dhq/vol/17/2/000673.xml,17,2,673,../data/dhq_data/000673.xml
48,/dhq/vol/17/2/000680.xml,17,2,680,../data/dhq_data/000680.xml
49,/dhq/vol/17/2/000680.xml,17,2,680,../data/dhq_data/000680.xml
102,/dhq/vol/16/4/000664.xml,16,4,664,../data/dhq_data/000664.xml
103,/dhq/vol/16/4/000664.xml,16,4,664,../data/dhq_data/000664.xml
152,/dhq/vol/16/2/000646.xml,16,2,646,../data/dhq_data/000646.xml
153,/dhq/vol/16/2/000646.xml,16,2,646,../data/dhq_data/000646.xml


In [154]:
merged_df['date_published'] = pd.to_datetime(merged_df['date_when'], errors='coerce')

In [155]:
missing

,DHQarticle-id,volume,issue,articleType,date_when,dhq_keywords,language_ident,dhq_abstract,file_name,title,authors,body_text,date
184,000680,17.0,2.0,article,None,DHQ classification scheme; full list available...,en,Southern Italian digital humanist Domenico Fio...,../data/dhq_data/000680.xml,"Language,\n Materiality, and Dig...","[{'author_name': 'Cristina Migliaccio', 'affi...",I shall try . . . to say something useful abou...,None
334,000646,16.0,2.0,article,None,DHQ classification scheme; full list available...,en,None,../data/dhq_data/000646.xml,Introduction: The\n Questions of...,"[{'author_name': 'Roopika Risam', 'affiliatio...",Minimal computing is the answer. Minimal compu...,None
459,000664,16.0,4.0,article,None,DHQ classification scheme; full list available...,en,"The annual, international Digital Humanities c...",../data/dhq_data/000664.xml,Response to\n\t\t\t\t\t\tThe circus we deserve...,[{'author_name': 'The Alliance of Digital Huma...,Executive Summary \n\t\t\t\t\tWe thank DHQ for...,None
498,000673,17.0,2.0,article,None,DHQ classification scheme; full list available...,en,"A novel, rule-based, automatic framework for i...",../data/dhq_data/000673.xml,Automatic\n Identification of Rh...,"[{'author_name': 'Heyam Abd Alhadi', 'affilia...","1. Introduction\n Rhetoric, balāgha...",None


In [156]:
for _, row in missing.iterrows():
    print(row.file_name)
    vol = row['volume']
    issue = row['issue']
    print(vol, issue)
    date = merged_df[(merged_df.volume == vol) & (merged_df.issue == issue)].sort_values(by='date_published', ascending=False).iloc[0].date_published
    print(date)
    merged_df.loc[(merged_df.file_name == row.file_name), 'date_published'] = date

../data/dhq_data/000680.xml
17.0 2.0
2023-07-20 00:00:00
../data/dhq_data/000646.xml
16.0 2.0
2022-06-25 00:00:00
../data/dhq_data/000664.xml
16.0 4.0
2022-10-14 00:00:00
../data/dhq_data/000673.xml
17.0 2.0
2023-07-20 00:00:00


In [157]:
merged_df.isna().sum()

DHQarticle-id      0
volume             0
issue              0
articleType        0
date_when          4
dhq_keywords       0
language_ident     2
dhq_abstract      21
file_name          0
title              0
authors            0
body_text          0
date               4
date_published     0
dtype: int64

In [158]:
merged_df.to_csv('../data/original_journal_datasets/dhq/dhq_data.csv', index=False)